# Comparison between different implementations for applying filter to a 3D volume

methods:
- Iterative mycarta's original implementation
- Vectorized Numpy
- Vecotrized Dask using numpy calls 
- Vecotrized Dask using dask calls 

In [1]:
from dask.distributed import Client # Used to create a LocalCluster on machine()
import numpy as np
from smallfoot import utils # holds tools

Hot reloading of libs mainly smallfoot

In [2]:
%load_ext autoreload
%autoreload 2

## Create the client cluster 

Here we create the dask cluster, using default params this can be played with but gives a good first approximation

In [3]:
Client()

Client Scheduler: tcp://127.0.0.1:60357 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


## Load in the data and create the filter

In [4]:
penobscot = np.load('images_and_data/penobscot.npy')
arr, _ = utils.pad_next_square_size(penobscot)

In [5]:
from skimage.draw import rectangle
from scipy import signal

def scipy_gaussian_2D(std):
    '''
    2D Gaussian filter kernel similar to astropy\'s Gaussian2DKernel
    (https://docs.astropy.org/en/stable/api/astropy.convolution.Gaussian2DKernel.html#astropy.convolution.Gaussian2DKernel)
    using scipy.signal.gaussian 
    (and inspired by https://gist.github.com/thomasaarholt/267ec4fff40ca9dff1106490ea3b7567)
    
    Parameters: 
    std (int) : standard deviation of the Gaussian in pixels
    
    Returns:
    out (2D array): 2D Gaussian filter kernel
    '''
    ksp1D = signal.gaussian(std*8+1, std)
    ksp2D = np.outer(ksp1D, ksp1D)
    ksp2D /= (2*np.pi*(std**2))
    return ksp2D

A = Ag = test1 = arr[:, :, 0]
rec= np.zeros(np.shape(A), dtype=np.uint8)
start1 = (np.shape(Ag)[0]//2+5,0)
end1 = (np.shape(Ag)[0]//2-5,np.shape(Ag)[0]//2-50)
rr1, cc1 = rectangle(start1, end=end1, shape=test1.shape)
rec[rr1, cc1] = 1

start2 = (np.shape(Ag)[0]//2+5,np.shape(Ag)[0]//2+50)
end2 = (np.shape(Ag)[0]//2-5,np.shape(Ag)[0]-1)
rr2, cc2 = rectangle(start2, end=end2, shape=test1.shape)
rec[rr2, cc2] = 1

gauss_kernel = scipy_gaussian_2D(11)
filter00 = signal.fftconvolve(rec, gauss_kernel, mode='same')
filter00 = utils.normalise(filter00)

# Comparison

In [9]:
%%timeit
utils.apply_filter_iterative(filter00, penobscot)

2.4 s ± 52.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
utils.apply_filter_vector(filter00, penobscot)

2.53 s ± 14.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
utils.apply_filter_vector_dask(filter00, penobscot).compute()

1.25 s ± 97.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
utils.apply_filter_vector_dask_true(filter00, penobscot).compute()

1.21 s ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
